In [1]:
%run launcher.py

In [2]:
install_libs()
show_projects()

No projects exist. Make a project first.


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': False, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, id='b3b565db-5eab-4365-b286-ab980543193f', precision=5)

VBox(children=(HBox(children=(Text(value='', description='Project Name', style=DescriptionStyle(description_width='initial')), Button(button_style='success', description='Add Project', disabled=True, icon='plus', style=ButtonStyle(), tooltip='Add a New Project'), Button(button_style='danger', description='Delete Project', icon='minus', style=ButtonStyle(), tooltip='Delete Project and its Files'))), HBox(children=(Text(value='', description='Duplicate Name', style=DescriptionStyle(description_width='initial')), Button(button_style='info', description='Duplicate Project', icon='plus', style=ButtonStyle(), tooltip='Duplicate Project and its Files')))))